In [179]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Solving Linear System


## Finding Lm Ln Polynomials 


In [180]:
def legendre_polynomial(n):
    y = sp.symbols('y')
    if n == 0:
        return sp.Lambda(y, 1)
    elif n == 1:
        return sp.Lambda(y, y)
    else:
        P_n_minus_1 = legendre_polynomial(n-1)
        P_n_minus_2 = legendre_polynomial(n-2)
        return sp.Lambda(y, ((2*n - 1) * y * P_n_minus_1(y) - (n - 1) * P_n_minus_2(y)) / n)

# Example usage:
n = 0
Ln = legendre_polynomial(n)
Ln


Lambda(y, 1)

## Computing Anm using GL Quadrature --> finding A 

In [181]:
n = [0]
m = [0]

def Anm(n, m):
    y = sp.symbols('y')
    Ln = legendre_polynomial(n)
    Lm = legendre_polynomial(m)
    return y * Ln(y) * Lm(y)

def Anm_gauleg(xi, ci, b , a, n2, n, m):
    sum = 0 
    Anm_expr = Anm(n,m)
    for i in range(n2):
        y_value = (0.5*(((b-a)*xi[i])+(b+a)))
        prod = ci[i]* Anm_expr.subs(sp.symbols('y'), y_value)
        sum = sum + prod 
    return sum 

def A_matrix(n,m):
    A_matrix = sp.zeros(len(n),len(m))
    n2 = math.floor((len(n)+len(m))/2) + 1
    xi = gl.gauleg(n2)[0] # satisfying dop of GL for p+1 is 2p + 1
    ci = gl.gauleg(n2)[1]
    for ni in n:
        for mi in m:
            A_matrix[ni ,mi] = Anm_gauleg(xi= xi, ci = ci, b = 1, a = -1, n2 = n2, n = ni, m = mi)

    return A_matrix

A = A_matrix(n,m)
A



Matrix([[0]])

## Define A(x,y) 

In [182]:
x = sp.symbols('x')
y = sp.symbols('y')

def a(x, y):
    return 2 + math.sin(x)


def a0():
    return 2 + sp.sin(x)


def a1():
    return 0 *x



## Define (S1) matrix and (S0) matrix

Error Indexing 

nodal index = (0,1,2,3,...,n) 

element index =(0,1,...,n-1) 

finite element function index =  nodal index = (0,1,2,3...) since phi_i(x_i) = 1 for all i

eg: nodal index = (0,1,2)

element index = (0,1)

element i = (x_i, x_i+1)

In [183]:

def GL_sym(xi, ci, y, x, n2, func):
	sum = 0 
	for i in range(n2):
		x_value = 0.5 * ((y - x) * xi[i] + (y + x))
		prod = ci[i] * func.subs(sp.symbols('x'), x_value)
		prod = prod.evalf()
		sum += prod 
	final = (y - x) / 2 * sum
	return final

def dphi_i_on_element(i, k, xlist):
    """
    Return the (constant) slope of the i-th shape function on the k-th subinterval
    [ x_k, x_{k+1} ] in a 1D mesh with nodes x_0 < ... < x_N.
    """
    if i == k:
        # node i is the left endpoint of the subinterval => slope from 1 at x_k down to 0 at x_{k+1}
        dx = xlist[k+1] - xlist[k]
        return -1.0 / dx
    elif i == k+1:
        # node i is the right endpoint => slope from 0 at x_k up to 1 at x_{k+1}
        dx = xlist[k+1] - xlist[k]
        return +1.0 / dx
    else:
        return 0.0


def phiij(numofnodes, i, j, xlist, func):
    finalsum = 0
    for k in range(numofnodes-1):
        x_left = xlist[k]
        x_right = xlist[k+1]

        # Derivatives of shape functions phi_i, phi_j on element k
        d_phi_i = dphi_i_on_element(i, k, xlist)
        d_phi_j = dphi_i_on_element(j, k, xlist)
        n2 = 5 
        xi = gl.gauleg(n2)[0]
        ci = gl.gauleg(n2)[1]
        onegl = GL_sym(xi = xi, ci = ci, y = x_right, x = x_left, n2 = 5, func = func) * d_phi_i *d_phi_j
        finalsum = finalsum + onegl
        
    return finalsum

def S1(func, mesh):
    I = list(range(1, len(mesh)-1))
    S1 = sp.zeros(len(I), len(I))
    target = func
    for local_i, g_i in enumerate(I):
            for local_j, g_j in enumerate(I):
                # 4) Evaluate your shape-function integral
                S1[local_i, local_j] = phiij(
                    numofnodes=len(mesh), 
                    i=g_i, 
                    j=g_j,
                    xlist=mesh,
                    func=func
                )

    return S1
initial_mesh = [0, 0.5, 1]
S1(func = a1(), mesh = initial_mesh)


Matrix([[0]])

In [184]:
# compute S0 

def S0(func, mesh):
    I = list(range(1, len(mesh)-1))
    S0 = sp.zeros(len(I), len(I))
    target = func
    for local_i, g_i in enumerate(I):
            for local_j, g_j in enumerate(I):
                # 4) Evaluate your shape-function integral
                S0[local_i, local_j] = phiij(
                    numofnodes=len(mesh), 
                    i=g_i, 
                    j=g_j,
                    xlist=mesh,
                    func= target
                )

    return S0

S0(func= a0(), mesh= initial_mesh)


Matrix([[9.83879077652744]])

## Define Kroneker Notation 


In [185]:
# n, m are integers, probably need to loop it for every possible n, m 

def delta_mn(m,n):
    if n == m:
        return 1 
    else:
        return 0 

def delta(n,m):
    delta = sp.zeros(len(n), len(m))
    for ni in n:
        for mi in m: 
            delta[ni,mi] = delta_mn(ni,mi)

    return delta 

delta(n,m)

Matrix([[1]])

## Kron Matrix A and S1 

In [186]:
def A_S1(matrixA, matrixS1):
    A_S1 = sp.kronecker_product(matrixA, matrixS1)
    return A_S1

A_S1(matrixA= A_matrix(n= n, m= m), matrixS1= S1(func = a1(), mesh = initial_mesh))


Matrix([[0]])

## Define S0 * delta_mn


In [187]:
def S0_delta(matrixS0, matrixdelta):
    return sp.kronecker_product(matrixS0 , matrixdelta) 

S0_delta(matrixS0=S0(func = a0(), mesh = initial_mesh), matrixdelta=delta(n,m))

Matrix([[9.83879077652744]])

## Define S(in)(jm)


In [188]:
def S_in_jm(matrixS1_Amn, matrixS0_delta):
    return matrixS0_delta + matrixS1_Amn

S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(func= a1(), mesh = initial_mesh)),
         matrixS0_delta=S0_delta(matrixS0=S0(func = a0(), mesh = initial_mesh), matrixdelta=delta(n,m)))

Matrix([[9.83879077652744]])

## Define Fi and F 

In [189]:
def phi(i, s, xlist):
    """
    Evaluate the piecewise-linear hat function phi_i at point s,
    given the node array xlist = [x_0, x_1, ..., x_N].
    """
    # Find which interval [x_k,x_{k+1}] contains s
    # If s < x_0 or s > x_N, phi_i(s)=0 outside domain
    if s < xlist[0] or s > xlist[-1]:
        return 0.0

    # Find k: x_k <= s < x_{k+1}
    k_found = None
    for k in range(len(xlist)-1):
        if xlist[k] <= s < xlist[k+1]:
            k_found = k
            break
    # handle s == xlist[-1]
    if s == xlist[-1]:
        k_found = len(xlist)-2

    # Cases:
    # 1) If i not in {k,k+1}, phi_i(s)=0 in that subinterval
    if i != k and i != k+1:
        return 0.0

    # 2) If i == k, then phi_i goes from 1 at x_k to 0 at x_{k+1}
    if i == k:
        dx = xlist[k+1] - xlist[k]
        # linear from phi_i(x_k)=1 to phi_i(x_{k+1})=0
        # slope = -1/dx
        return 1.0 - (s - xlist[k]) / dx

    # 3) If i == k+1, then phi_i goes from 0 at x_k to 1 at x_{k+1}
    elif i == k+1:
        dx = xlist[k+1] - xlist[k]
        return (s - xlist[k]) / dx
    
def integrand_for_force(i, xlist, f, xi, ci, n2):
    """
    Compute \int f(s)*phi_i(s) ds over the entire domain,
    by summing subinterval integrals or a single Gauss-Legendre over [0,1].
    But typically we do subinterval-based integration for a non-uniform mesh.
    """
    # We'll do an element-by-element integration approach:
    total = 0.0
    for k in range(len(xlist)-1):
        x_left = xlist[k]
        x_right = xlist[k+1]
        # do Gauss-Legendre on each subinterval
        sum_local = 0.0
        for j in range(n2):
            s = 0.5*((x_right - x_left)*xi[j] + (x_left + x_right))
            w = ci[j]
            val = f(s)*phi(i, s, xlist)
            sum_local += w*val
        total += 0.5*(x_right - x_left)*sum_local
    return total

def f(s): 
    return 4 - math.cos(s)  + 2*s*math.cos(s) + 2* math.sin(s)


def build_force_vector(num_nodes, xlist, f):
    """
    Build the vector F of length num_nodes where
    F[i] = \int f(x)*phi_i(x) dx
    """
    interior = list(range(1, len(xlist)-1))
    num_interior = len(interior)

    # 2) Create a vector of that size
    F_interior = sp.zeros(num_interior, 1)

    # 3) Gauss-Legendre setup
    n2 = 5
    xi, ci = gl.gauleg(n2)

    # 4) Fill each entry for the interior node i
    for local_i, global_i in enumerate(interior):
        val = integrand_for_force(global_i, xlist, f, xi, ci, n2)
        F_interior[local_i, 0] = val

    return F_interior

build_force_vector(num_nodes=len(initial_mesh), xlist = initial_mesh, f = f)

Matrix([[2.44987501866166]])

## Solving for C 

In [190]:
def C(matrixS, matrixF):
    return matrixS.inv() * matrixF 

In [191]:
C_test = C(matrixS= S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(func= a1(), mesh = initial_mesh)),
     matrixS0_delta=S0_delta(matrixS0=S0(func = a0(), mesh = initial_mesh), matrixdelta=delta(n,m))), 
     matrixF= build_force_vector(num_nodes=len(initial_mesh), xlist = initial_mesh, f = f))

C_test

Matrix([[0.249001637935667]])

In [ ]:
def solve_scF_once(n_list, m_list, i_list):
    """
    Build and solve the system S*C = F for a single iteration.

    Parameters
    ----------
    n_list, m_list : lists of indices (e.g. polynomial degrees)
    i_list         : indices for the piecewise-constant or piecewise-linear basis

    Returns
    -------
    C : Sympy Matrix
        The solution vector for the unknowns.
    """
    # 1) Build the big matrix S by combining (S0_delta + S1_Amn)
    #    i.e. S = matrixS0_delta + matrixS1_Amn
    #    which depends on A_matrix(...), S0(...), S1(...), etc.
    A = A_matrix(n_list, m_list)  # build the “A” part (possibly a small matrix)
    Delta = delta(n_list, m_list) # your delta for Kronecker product
    # S1(...) and S0(...) might need i_list & a0() or a1() or a custom func 
    S1_mat = S1(listi=i_list, listj=i_list, func=a1())  
    S0_mat = S0(listi=i_list, listj=i_list, func=a0())  

    # Build the big Kronecker products
    matrixS1_Amn = A_S1(A, S1_mat)           # = kronecker(A, S1_mat)
    matrixS0_delta = S0_delta(S0_mat, Delta) # = kronecker(S0_mat, Delta)
    S = S_in_jm(matrixS1_Amn, matrixS0_delta) # S = matrixS0_delta + matrixS1_Amn

    # 2) Build the right-hand side vector F
    Fvec = F(n_list, i_list)

    # 3) Solve S*C = F for C
    C_sol = C(S, Fvec)  # calls your function C(...) which does: S_inv = S.inv(); C = S_inv*F

    return C_sol

solve_scF_once(n_list=n, m_list=m, i_list= ijlist)

TypeError: S1() got an unexpected keyword argument 'listi'

## Computing Initial Energy Norm 

In [203]:
def approx_new(c, F):
    approx_new = c.T *F

    return approx_new[0,0]

approx = approx_new(c = C( matrixS= S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(func= a1(), mesh = initial_mesh)), matrixS0_delta=S0_delta(matrixS0=S0(func = a0(), mesh = initial_mesh), matrixdelta=delta(n,m))),
                                 matrixF= build_force_vector(num_nodes=len(initial_mesh), xlist = initial_mesh, f = f)),
                                 F = build_force_vector(num_nodes=len(initial_mesh), xlist = initial_mesh, f = f))

def energy_norm(approximation):
    actual = 0.8146667469752307
    energy_norm = actual - approximation
    return energy_norm

print(energy_norm(approx))
print(approx)

0.204643854590804
0.610022892384427


Therefore, We have shown that as n, m, l increases, the error decreases 


## Computing element error r(x)


In [31]:
def piecewise_linear_derivative(x_num, mesh, uh):
    """
    Return the piecewise-constant slope u_h'(x_num) for x_num in an element
    of the 1D mesh, or 0.0 if x_num is outside [mesh[0], mesh[-1]].
    """
    m = len(mesh) - 1
    if x_num < mesh[0] or x_num > mesh[m]:
        # outside the domain
        return 0.0
    
    # Find k so that mesh[k] <= x_num < mesh[k+1]
    # (or x_num == mesh[m], in which case we set k = m-1)
    k_found = None
    for k in range(m):
        if mesh[k] <= x_num < mesh[k+1]:
            k_found = k
            break
    if x_num == mesh[m]:
        k_found = m-1
    
    dx = mesh[k_found+1] - mesh[k_found]
    dy = uh[k_found+1] - uh[k_found]
    return dy / dx


def assemble_nodal_values(C):
    """
    Given:
      C = [U_1, U_2, ..., U_{N-1}]  from the system solve
    Returns:
      U_full = [U_0, U_1, U_2, ..., U_{N-1}, U_N]  (with BCs)
    """
    N_minus_1 = len(C)          # number of interior unknowns
    U_full = [0]*(N_minus_1+2)  # +2 for endpoints
    U_full[0] = 0.0            # Dirichlet BC at x_0
    U_full[-1] = 0.0           # Dirichlet BC at x_N
    
    # Fill in interior:
    for i in range(N_minus_1):
        U_full[i+1] = C[i]
    
    return U_full



In [159]:
# label each finite element for error calculation
initial_mesh = [0, 0.5, 1]
initial_nodal = assemble_nodal_values(C_test)

def epsilon_k_sq():
    pass
     

def f_expr(x_num): ## is f(s) supposed to be a function of y? --> u(x,y) and a(x,y) = f(x,y)
    return -4 + math.cos(x_num) - 2*x_num*math.cos(x_num) - 2*math.sin(x_num)

def r(x_num, mesh, uh):
    """
    Residual r(x) = f(x) + u_h'(x)*a'(x), but with a'(x)=cos(x) if a(x)=2+sin(x).
    => r(x) = f(x) + slope*cos(x_num).
    """
    slope = piecewise_linear_derivative(x_num, mesh, uh)
    return f_expr(x_num) + slope*math.cos(x_num)

def GL(xi, ci, x_right, x_left, n2, func):
	sum = 0 
	for i in range(n2):
		prod = ci[i]*func(0.5*(((x_right-x_left)*xi[i])+(x_right+x_left)))
		sum = sum + prod 
	return (x_right-x_left)/2 * (sum)



def element_residual_l2(x_left_element_i, x_right_element_i, mesh, uh):
    """
    Carry out Gauss-Legendre Quadrature of [r(x)]^2 on the element
    [x_left_element_i, x_right_element_i], where
      r(x) = f(x) + (u_h'(x))*cos(x).
    We assume piecewise_linear_derivative + f_expr are already defined.
    """
    def integrand_interior(x_val):
        # We want [r(x_val)]^2
        return (r(x_val, mesh, uh))**2
    
    n2 = 5
    xi, ci = gl.gauleg(n2)  # your routine that returns points & weights
    # Integrate (r(x))^2 over this specific element
    r_sq = GL(
        xi=xi, ci=ci,
        x_right=x_right_element_i,
        x_left=x_left_element_i,
        n2=n2,
        func=integrand_interior
    )
    return r_sq

In [ ]:
# Let's compute the L2-residual on element [0,0.5]
x_left = initial_mesh[0]
x_right = initial_mesh[1]
res_sq_first = element_residual_l2(x_left, x_right, initial_mesh, initial_nodal)
print("∫[0,0.5] [r(x)]^2 dx =", res_sq_first)

∫[0,0.5] [r(x)]^2 dx = 6.79777179643858


## Calculate boundary error 

In [161]:
def slope_at_node(mesh, uh, i, side):
    """
    Return the slope of the piecewise-linear solution u_h on the element
    *adjacent* to node i from the specified side ('left' or 'right').

    Parameters
    ----------
    mesh : list of floats, [x_0, x_1, ..., x_m]
           The node coordinates (strictly increasing).
    uh   : list of floats, [u_0, u_1, ..., u_m]
           The nodal values of u_h at mesh[i].
    i    : int
           The index of the node x_i in mesh.
    side : str
           Either 'left' or 'right'.
    
    Returns
    -------
    slope : float
        The slope in the element to the left or right of x_i.
        If x_i is at a boundary (e.g. i=0 and side='left'),
        we either raise an error or return 0.0, depending on your convention.
    """
    # Number of intervals is len(mesh)-1
    n = len(mesh) - 1

    if side == 'left':
        # The element to the left of node x_i is [x_{i-1}, x_i].
        # So we must have i >= 1 to have a left neighbor.
        if i == 0:
            slope = 0 
        else:
            dx = mesh[i] - mesh[i-1]
            dy = uh[i]   - uh[i-1]
            slope = dy/dx
        return slope
    
    elif side == 'right':
        # The element to the right of node x_i is [x_i, x_{i+1}].
        # So we must have i <= n-1 to have a right neighbor.
        if i >= n:
            slope = 0 
        else:
            dx = mesh[i+1] - mesh[i]
            dy = uh[i+1]  - uh[i]
            slope = dy/dx
        return slope

    else:
        raise ValueError("side must be 'left' or 'right'.")



In [162]:
def j(mesh, uh, i):
    x_num = mesh[i]
    slope_right = slope_at_node(mesh, uh, i, 'right')
    sigma_right = 1 * a0().subs(x, x_num).evalf() *slope_right
    slope_left = slope_at_node(mesh, uh, i , 'left')
    sigma_left = -1 * a0().subs(x, x_num).evalf() * slope_left 
    j_x = sigma_right - sigma_left
    return j_x


In [222]:
def mesh_size(mesh, i):
    return mesh[i+1] - mesh[i]
    


def sum_of_error(i, mesh, nodal):
    x_left_element_i= mesh[i]
    x_right_element_i = mesh[i+1]
    h = x_right_element_i - x_left_element_i
    residual_sq = element_residual_l2(x_left_element_i, x_right_element_i, nodal, i)
    boundary_sq = j(mesh, nodal, i=i+1)**2
    return h ** 2 * residual_sq + h * boundary_sq
    

def sum_of_error_list(mesh, nodal):
    error_list = []
    for i in list(range(len(mesh) -1)):
        error = sum_of_error(i = i, mesh = mesh, nodal = nodal)
        error_list.append(error)
    return error_list

error_0 = sum_of_error(i = 0, mesh = initial_mesh, nodal = initial_nodal)  
print("Error at element 0" , error_0)
error_1 = sum_of_error(i = 1, mesh = initial_mesh, nodal = initial_nodal)
print("Error at element 1" , error_1)



Error at element 0 2.80756959758409
Error at element 1 4.11705961079297


```
def adaptive_loop():
    # Initialize
    n_list, m_list, i_list = initial_data()
    for iteration in range(max_iterations):
        # 1) Solve
        current_C = solve_scF_once(n_list, m_list, i_list)
        
        # 2) Estimate error
        error_value = compute_error(current_C, n_list, m_list, i_list)
        
        # 3) Heap sort 
            if error_i > k * total error:
            add midpoint node to mesh index. 
        
        # 4) Mark for refinement if error too large
        for 
        if error_value < tolerance:
            print("Converged!")
            break
        
        # 5) Refine (e.g. split the mesh, update i_list, etc.)
        n_list, m_list, i_list = refine(n_list, m_list, i_list, current_C, error_value)

    return current_C

```


In [223]:
def solve_scF_once(n_list, m_list, mesh):
    """
    Build and solve the system S*C = F for a single iteration.

    Parameters
    ----------
    n_list, m_list : lists of indices (e.g. polynomial degrees)
    i_list         : indices for the piecewise-constant or piecewise-linear basis

    Returns
    -------
    C : Sympy Matrix
        The solution vector for the unknowns.
    """

    # Build the big Kronecker products
    matrixS1_Amn = A_S1(matrixA=A_matrix(n,m), matrixS1=S1(func= a1(), mesh = mesh))          # = kronecker(A, S1_mat)
    matrixS0_delta = S0_delta(matrixS0=S0(func = a0(), mesh = mesh), matrixdelta=delta(n,m))
    S = S_in_jm(matrixS1_Amn, matrixS0_delta) # S = matrixS0_delta + matrixS1_Amn

    # 2) Build the right-hand side vector F
    Fvec = build_force_vector(num_nodes=len(mesh), xlist = mesh, f = f)

    # 3) Solve S*C = F for C
    C_sol = C(S, Fvec)  # calls your function C(...) which does: S_inv = S.inv(); C = S_inv*F

    return C_sol, Fvec

solve_scF_once(n_list=n, m_list=m, mesh = initial_mesh)[1]

Matrix([[2.44987501866166]])

In [224]:
# criteria for refinement 
# initialize upper bound for termination 

epsilon = 1
def refine_mesh(mesh, element_index):
    """
    Refine the element [mesh[element_index], mesh[element_index+1]]
    by bisection. That is, insert a midpoint and return the new mesh.

    Example:
      mesh = [0, 0.5, 1], element_index=1 => the element [0.5, 1].
      We compute midpoint = 0.75, so new mesh = [0, 0.5, 0.75, 1].
    """
    x_left = mesh[element_index]
    x_right = mesh[element_index+1]
    midpoint = 0.5*(x_left + x_right)

    # Insert the midpoint after mesh[element_index]
    return mesh[:element_index+1] + [midpoint] + mesh[element_index+1:]

def element_selection(mesh, errors, epsilon):
    """
    Given a 1D mesh (list of node coords) and a list 'errors'
    of elementwise errors, refine any element whose error is > epsilon
    by bisection. Return the new mesh.
    """
    new_mesh = mesh[:]  # copy
    # We gather all elements to refine (descending order so indexing is stable).
    # Or just pick the largest error — depends on your marking strategy.
    elements_to_refine = []
    for elem_index, err_value in enumerate(errors):
        if err_value > epsilon:
            elements_to_refine.append(elem_index)

    # Sort in descending order so we refine from rightmost to leftmost,
    # avoiding index shifting.
    elements_to_refine.sort(reverse=True)

    return elements_to_refine

def element_refinement(mesh, element_index):
    new_mesh = mesh[:]
    for elem_idx in element_index:
        new_mesh = refine_mesh(new_mesh, elem_idx)

    return new_mesh

In [ ]:
#solving loop

def refinement_loop(epsilon):
    '''
    Constructs the whole loop of 
    1. Solving Sc = F with a given initial mesh
    2. Error computation --> outputs a list of error given a mesh 
        -> breaks when every error element is < epsilon 
        -> to do this we check length of refinement list = 0 
    3. refinement by bisection --> outputs a list of new mesh
    4. Solving Sc = F and update energy norm 
    
    '''
    # 1) Solving Sc = F 
    initial_mesh = [0, 0.5, 1]
    c_sol = solve_scF_once(n_list=n, m_list=m, mesh=initial_mesh)[0]
    
    # 2) Error computation 
    errors = sum_of_error_list(mesh=initial_mesh)
        # Error selection for refinement 
    elements_to_refine = element_selection(mesh = initial_mesh, epsilon=epsilon)
    new_mesh = element_refinement(elements_to_refine)

    k = 1
    while k > 0 : 
        # Resolve Sc = F given new mesh 
        c_sol = solve_scF_once(n_list=n, m_list=m, mesh = new_mesh)[0]
        f_sol = solve_scF_once(n_list=n, m_list=m, mesh = new_mesh)[0]
        errors = sum_of_error_list(mesh = new_mesh)
         # Error selection for refinement after resolve
        elements_to_refine = element_selection(mesh = initial_mesh, epsilon=epsilon)
        new_mesh = element_refinement(elements_to_refine)
        # Update k 
        k = len(elements_to_refine)
        approximation = approx_new(c_sol, f_sol)
        energy_norm = energy_norm(approximation)

    return energy_norm
    

    


In [229]:
def refinement_loop(epsilon):
    """
    1) Start with initial mesh
    2) Solve once
    3) Estimate errors
    4) If all errors < epsilon, done. Else refine, go back to step 2.
    """
    mesh = [0, 0.5, 1]

    while True:
        # Solve for c_sol on the current mesh
        c_sol = solve_scF_once(n_list=n, m_list=m, mesh=mesh)[0]
        f_sol = solve_scF_once(n_list=n, m_list=m, mesh=mesh)[1]
        nodal = assemble_nodal_values(c_sol)
        # Compute the elementwise errors (list of length = #elements in 'mesh')
        errors = sum_of_error_list(mesh= mesh, nodal = nodal)

        # Mark which elements to refine
        elements_to_refine = element_selection(mesh= mesh, errors= errors, epsilon= epsilon)

        if not elements_to_refine:
            # No elements exceed threshold => we are done
            break

        # Otherwise, refine those elements
        mesh = element_refinement(mesh, elements_to_refine)
        print(mesh)

    # after the loop, final solution is c_sol on the final mesh
    approximation = approx_new(c = c_sol, F = f_sol)
    enorm = energy_norm(approximation)
    return mesh, c_sol, enorm

refinement_loop(epsilon=epsilon)


[0, 0.25, 0.5, 0.75, 1]
[0, 0.125, 0.25, 0.5, 0.625, 0.75, 0.875, 1]
[0, 0.0625, 0.125, 0.25, 0.5, 0.625, 0.75, 0.8125, 0.875, 1]
[0, 0.0625, 0.125, 0.25, 0.5, 0.625, 0.75, 0.8125, 0.84375, 0.875, 1]


([0, 0.0625, 0.125, 0.25, 0.5, 0.625, 0.75, 0.8125, 0.84375, 0.875, 1],
 Matrix([
 [0.0586856842100787],
 [ 0.109556206016354],
 [ 0.187743010839068],
 [ 0.249604709731213],
 [ 0.234050597233757],
 [ 0.187252788880058],
 [ 0.152169953924309],
 [ 0.131702433175406],
 [ 0.109281527008771]]),
 0.0195348062681918)